In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, T5Tokenizer

In [3]:
# directory where to cache models
cache_dir = '/home/jupyter/data/transformers'
checkpoint = "lmsys/fastchat-t5-3b-v1.0"

In [4]:
# loading model and tokenizer based on particular model checkpoint
# tokenizer is used to preprocess text input in a way that the model can understand
tokenizer = T5Tokenizer.from_pretrained(checkpoint, device_map="auto", use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="auto", low_cpu_mem_usage=True, 
                                              cache_dir=cache_dir, torch_dtype=torch.float16)

In [5]:
system="""A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.
### Human: What are the key differences between renewable and non-renewable energy sources?
### Assistant: Renewable energy sources are those that can be replenished naturally in a relatively short amount of time, such as solar, wind, hydro, geothermal, and biomass. Non-renewable energy sources, on the other hand, are finite and will eventually be depleted, such as coal, oil, and natural gas. Here are some key differences between renewable and non-renewable energy sources:
1. Availability: Renewable energy sources are virtually inexhaustible, while non-renewable energy sources are finite and will eventually run out.
2. Environmental impact: Renewable energy sources have a much lower environmental impact than non-renewable sources, which can lead to air and water pollution, greenhouse gas emissions, and other negative effects.
3. Cost: Renewable energy sources can be more expensive to initially set up, but they typically have lower operational costs than non-renewable sources.
4. Reliability: Renewable energy sources are often more reliable and can be used in more remote locations than non-renewable sources.
5. Flexibility: Renewable energy sources are often more flexible and can be adapted to different situations and needs, while non-renewable sources are more rigid and inflexible.
6. Sustainability: Renewable energy sources are more sustainable over the long term, while non-renewable sources are not, and their depletion can lead to economic and social instability."""

system = system + "\n"


def generate(prompt, max_new_tokens=1024, temperature=0.7):
    torch.cuda.empty_cache()

    template = system + "### Human: %s \n"
    input_start = template % prompt
    input_final = template % prompt + "### Assistant:"
    
    inputs = tokenizer(input_final, return_tensors="pt", max_length=4680, truncation="longest_first").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, 
                             temperature=temperature)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True, spaces_between_special_tokens=False)
        
    return result[0]

# CNBC

In [6]:
import pandas as pd


cnbc_df = pd.read_excel("gs://scraped-news-article-data-null/Data_CNBC.xlsx")
cnbc_df.head()

,Source,Date,Title,Author,Text,Summary
0,https://www.cnbc.com/2023/05/02/pfizer-pfe-q1-...,MAY 2 2023,Pfizer earnings and revenue top expectations d...,Annika Kim Constantino,Pfizer\n on Tuesday reported first-quarter rev...,Pfizer reported revenue and adjusted earnings ...
1,https://www.cnbc.com/2023/05/01/scientists-dev...,MAY 1 2023,Scientists develop A.I. system focused on turn...,Ashley Capoot,Scientists have developed a noninvasive AI sys...,Scientists have developed a noninvasive AI sys...
2,https://www.cnbc.com/2023/04/30/alzheimers-tre...,APR 30 2023,Alzheimer’s patients may wait years to get tre...,Spencer Kimball,Seniors with early Alzheimer’s disease will fa...,A shortage of dementia specialists and testing...
3,https://www.cnbc.com/2023/05/02/gilead-governm...,MAY 2 2023,Gilead Sciences battles U.S. government in cou...,Spencer Kimball,Gilead Sciences\n and the U.S. government face...,The U.S. is trying to enforce CDC patents on a...
4,https://www.cnbc.com/2023/05/01/states-medicar...,MAY 1 2023,Bipartisan attorneys general call on Medicare ...,Spencer Kimball,Democratic and Republican attorneys general in...,"Attorneys general from 23 states, the District..."


In [7]:
cnbc_relevant = cnbc_df[["Source", "Text"]]
cnbc_relevant.head()

,Source,Text
0,https://www.cnbc.com/2023/05/02/pfizer-pfe-q1-...,Pfizer\n on Tuesday reported first-quarter rev...
1,https://www.cnbc.com/2023/05/01/scientists-dev...,Scientists have developed a noninvasive AI sys...
2,https://www.cnbc.com/2023/04/30/alzheimers-tre...,Seniors with early Alzheimer’s disease will fa...
3,https://www.cnbc.com/2023/05/02/gilead-governm...,Gilead Sciences\n and the U.S. government face...
4,https://www.cnbc.com/2023/05/01/states-medicar...,Democratic and Republican attorneys general in...


In [8]:
print(cnbc_relevant.iloc[0, 1])

Pfizer
 on Tuesday reported first-quarter revenue and adjusted earnings that topped Wall Street’s expectations, despite a decline in sales driven by the lower demand for the company’s Covid vaccine.

The pharmaceutical giant’s stock closed slightly lower Tuesday at $39.06. Shares are down more than 23% year to date through Tuesday’s close, putting the company’s market value at around $220.47 billion. 

Here’s what Pfizer reported compared with Wall Street’s expectations, based on a survey of analysts by Refinitiv:

Earnings per share: $1.23 adjusted vs. 98 cents expected 
Revenue: $18.28 billion vs. $16.59 billion expected
Pfizer’s net income of $5.54 billion, or 97 cents per share, fell from $7.86 billion, or $1.37 per share, during the year-earlier quarter.

The company reported first-quarter sales of $18.28 billion, down 29% over the same period a year ago.

Covid-related sales contributed $7.1 billion to that number. Pfizer raked in $3 million in Covid vaccine revenue and $4 millio

In [9]:
prompt = "You are an equity researcher specialized in the health and pharmaceutical industry. Write a summary for the following article. Focus on the information that will impact investment decisions on the companies mentioned.\n"
article = cnbc_relevant.iloc[0, 1]
full_prompt = prompt + article
print(generate(full_prompt, temperature=0))

Pfizer reported first-quarter revenue and adjusted earnings that topped Wall Street's expectations, despite a decline in sales driven by the lower demand for the company's Covid vaccine. The company reported first-quarter sales of $18.28 billion, down 29% over the same period a year ago. Covid-related sales contributed $7.1 billion to that number. Pfizer maintained its 2023 sales forecast of $67 billion to $71 billion. The company reaffirmed its forecast of $13.5 billion in Covid vaccine sales in 2023 and $8 billion in revenue for Paxlovid. Pfizer CEO Albert Bourla said during the earnings call that the company expects 2023 to be a "transition year" for Covid sales as the U.S. pivots to the commercial market for Covid products. Sales of Paxlovid increased $2.8 billion year over year amid new launches in certain international markets and strong demand in China due to increased Covid cases. Pfizer expects higher uptake of Paxlovid after this year. Pfizer also expects U.S. uptake of its C

In [10]:
summary = generate(full_prompt, temperature=0)
print(summary)

Pfizer reported first-quarter revenue and adjusted earnings that topped Wall Street's expectations, despite a decline in sales driven by the lower demand for the company's Covid vaccine. The company reported first-quarter sales of $18.28 billion, down 29% over the same period a year ago. Covid-related sales contributed $7.1 billion to that number. Pfizer maintained its 2023 sales forecast of $67 billion to $71 billion. The company reaffirmed its forecast of $13.5 billion in Covid vaccine sales in 2023 and $8 billion in revenue for Paxlovid. Pfizer CEO Albert Bourla said during the earnings call that the company expects 2023 to be a "transition year" for Covid sales as the U.S. pivots to the commercial market for Covid products. Sales of Paxlovid increased $2.8 billion year over year amid new launches in certain international markets and strong demand in China due to increased Covid cases. Pfizer expects higher uptake of Paxlovid after this year. Pfizer also expects U.S. uptake of its C

In [11]:
cnbc_score = cnbc_relevant[["Source"]].copy()
cnbc_score["Accuracy"] = 1
cnbc_score["Relevancy"] = 1
cnbc_score["Summary"] = ""
cnbc_score["Comment"] = ""
cnbc_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.cnbc.com/2023/05/02/pfizer-pfe-q1-...,1,1,,
1,https://www.cnbc.com/2023/05/01/scientists-dev...,1,1,,
2,https://www.cnbc.com/2023/04/30/alzheimers-tre...,1,1,,
3,https://www.cnbc.com/2023/05/02/gilead-governm...,1,1,,
4,https://www.cnbc.com/2023/05/01/states-medicar...,1,1,,


In [12]:
cnbc_score.loc[0, "Relevancy"] = 4
cnbc_score.loc[0, "Summary"] = summary
cnbc_score.loc[0, "Comment"] = "Did not go into detail about the specific mergers and the RSV vaccine"
cnbc_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.cnbc.com/2023/05/02/pfizer-pfe-q1-...,1,4,Pfizer reported first-quarter revenue and adju...,Did not go into detail about the specific merg...
1,https://www.cnbc.com/2023/05/01/scientists-dev...,1,1,,
2,https://www.cnbc.com/2023/04/30/alzheimers-tre...,1,1,,
3,https://www.cnbc.com/2023/05/02/gilead-governm...,1,1,,
4,https://www.cnbc.com/2023/05/01/states-medicar...,1,1,,


In [13]:
print(cnbc_relevant.iloc[1, 1])

Scientists have developed a noninvasive AI system focused on translating a person’s brain activity into a stream of text, according to a peer-reviewed study published Monday in the journal Nature Neuroscience.

The system, called a semantic decoder, could ultimately benefit patients who have lost their ability to physically communicate after suffering from a stroke, paralysis or other degenerative diseases. Researchers at the University of Texas at Austin developed the system in part by using a transformer model, which is similar to those that support Google’s chatbot Bard and OpenAI’s chatbot ChatGPT. 

The study’s participants trained the decoder by listening to several hours of podcasts within an fMRI scanner, which is a large piece of machinery that measures brain activity. The system requires no surgical implants. Once the AI system is trained, it can generate a stream of text when the participant is listening to or imagines telling a new story. The resultant text is not an exact 

In [14]:
article = cnbc_relevant.iloc[1, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

The article discusses a new AI system developed by scientists at the University of Texas at Austin that can translate a person's brain activity into a stream of text. The system, called a semantic decoder, could benefit patients who have lost their ability to physically communicate after suffering from a stroke, paralysis, or other degenerative diseases. The system is noninvasive and requires no surgical implants. The trained system produces text that closely or precisely matches the intended meaning of the participant's original words around half of the time. The researchers believe the system could eventually be used via more portable brain-imaging systems. The leading researchers of the study have filed a PCT patent application for the technology.



In [15]:
cnbc_score.loc[1, "Relevancy"] = 4
cnbc_score.loc[1, "Summary"] = summary
cnbc_score.loc[1, "Comment"] = "Did not mention that the system is only in lab currently. Did mention that a patent is coming up."
cnbc_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.cnbc.com/2023/05/02/pfizer-pfe-q1-...,1,4,Pfizer reported first-quarter revenue and adju...,Did not go into detail about the specific merg...
1,https://www.cnbc.com/2023/05/01/scientists-dev...,1,4,The article discusses a new AI system develope...,Did not mention that the system is only in lab...
2,https://www.cnbc.com/2023/04/30/alzheimers-tre...,1,1,,
3,https://www.cnbc.com/2023/05/02/gilead-governm...,1,1,,
4,https://www.cnbc.com/2023/05/01/states-medicar...,1,1,,


In [16]:
print(cnbc_relevant.iloc[2, 1])

Seniors with early Alzheimer’s disease will face major hurdles to get treated even if promising new drugs roll out more broadly in the coming years, putting them at risk of developing more severe disease as they wait months or perhaps years for a diagnosis.

The U.S. health-care system is not currently prepared to meet the needs of an aging population in which a growing number of people will need to undergo evaluation for Alzheimer’s, according to neurologists, health policy experts and the companies developing the drugs.

There are not enough dementia specialists or the needed testing capacity in the U.S. to diagnose everyone who may benefit from a new treatment like Eisai
 and Biogen
’s Leqembi. After patients are diagnosed, the capacity may not exist — at least initially — to provide the twice monthly intravenous infusions for everyone who is eligible.

Researchers estimate that the wait time from the initial evaluation to the confirmatory diagnostic tests to the infusions could ran

In [17]:
article = cnbc_relevant.iloc[2, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

 capacity to provide the twice-monthly infusions, according to a study published in the journal Alzheimer’s and Dementia. The U.S. has an estimated 4,000 infusion centers, but only about a third of them are equipped to administer intravenous drugs, according to a study published in the journal Alzheimer’s and Dementia. The number of infusion centers is expected to grow as the U.S. population ages, but it’s not clear how quickly they will be able to handle the expected surge in patients seeking treatment.
The U.S. health-care system is not currently prepared to meet the needs of an aging population in which a growing number of people will need to undergo evaluation for Alzheimer’s, according to neurologists, health policy experts and the companies developing the drugs. There are not enough dementia specialists or the needed testing capacity in the U.S. to diagnose everyone who may benefit from a new treatment like Eisai and Biogen’s Leqembi. After patients are diagnosed, the wait time f

In [18]:
cnbc_score.loc[2, "Relevancy"] = 1
cnbc_score.loc[2, "Summary"] = summary
cnbc_score.loc[2, "Comment"] = "Ran out of context window"
cnbc_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.cnbc.com/2023/05/02/pfizer-pfe-q1-...,1,4,Pfizer reported first-quarter revenue and adju...,Did not go into detail about the specific merg...
1,https://www.cnbc.com/2023/05/01/scientists-dev...,1,4,The article discusses a new AI system develope...,Did not mention that the system is only in lab...
2,https://www.cnbc.com/2023/04/30/alzheimers-tre...,1,1,capacity to provide the twice-monthly infusio...,Ran out of context window
3,https://www.cnbc.com/2023/05/02/gilead-governm...,1,1,,
4,https://www.cnbc.com/2023/05/01/states-medicar...,1,1,,


In [19]:
print(cnbc_relevant.iloc[3, 1])

Gilead Sciences
 and the U.S. government faced off in court Tuesday in the first day of a trial that will probe allegations that the drugmaker violated patents for a crucial HIV prevention drug regimen.

The U.S. is trying to enforce four patents issued to the Centers for Disease Control and Prevention on a two-drug regimen known as pre-exposure prophylaxis, or PrEP for short. The government accuses Gilead of reaping billions of dollars in PrEP sales without paying royalties to the CDC.

The U.S. filed the lawsuit against Gilead in 2019. Gilead has rejected U.S. allegations that the company’s sales of its PrEP oral medications, Truvada and Descovy, infringe on any CDC patents.

The trial in Delaware federal district court is expected to last six days.

Scientists at the CDC discovered in the mid-2000s that two drugs, emtricitabine and tenofovir, taken together were highly effective in preventing HIV infection, according to U.S. government’s lawsuit.

Gilead’s Truvada and Descovy both c

In [20]:
article = cnbc_relevant.iloc[3, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

The U.S. government is suing Gilead Sciences for alleged patent infringement on a two-drug HIV prevention regimen known as pre-exposure prophylaxis (PrEP). The U.S. government claims that Gilead has reaped billions of dollars in PrEP sales without paying royalties to the CDC. The trial in Delaware federal district court is expected to last six days.
The U.S. government claims that Gilead has repeatedly refused to obtain a license from the CDC to use the patented regimens. The company has rejected U.S. allegations that its sales of PrEP oral medications, Truvada and Descovy, infringe on any CDC patents. The two-drug PrEP regimen has played a key part in reducing new HIV infections in communities that face a higher risk from the virus, such as men who have sex with men, after decades of failed efforts to develop a vaccine. Clinical trials have demonstrated that PrEP is 99% effective at preventing HIV infection.



In [21]:
cnbc_score.loc[3, "Relevancy"] = 5
cnbc_score.loc[3, "Summary"] = summary
cnbc_score.loc[3, "Comment"] = "Covers the lawsuit and background information that may be unknown to retail investor"
cnbc_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.cnbc.com/2023/05/02/pfizer-pfe-q1-...,1,4,Pfizer reported first-quarter revenue and adju...,Did not go into detail about the specific merg...
1,https://www.cnbc.com/2023/05/01/scientists-dev...,1,4,The article discusses a new AI system develope...,Did not mention that the system is only in lab...
2,https://www.cnbc.com/2023/04/30/alzheimers-tre...,1,1,capacity to provide the twice-monthly infusio...,Ran out of context window
3,https://www.cnbc.com/2023/05/02/gilead-governm...,1,5,The U.S. government is suing Gilead Sciences f...,Covers the lawsuit and background information ...
4,https://www.cnbc.com/2023/05/01/states-medicar...,1,1,,


In [22]:
print(cnbc_relevant.iloc[4, 1])

Democratic and Republican attorneys general in nearly half of U.S. states are calling on Medicare to provide unrestricted coverage of antibody treatments for Alzheimer’s disease, according to a letter released Monday.

The push by attorneys general from 23 states, the District of Columbia, and two U.S. territories adds to mounting pressure on the federal Centers for Medicare and Medicaid Services, or CMS, to end a controversial policy that severely restricts access to new drugs such as Eisai and Biogen
’s Leqembi.

Twice-monthly infusions of Leqembi have shown promise in slowing progression of early Alzheimer’s to more advanced stages of the mind-wasting disease. Medicare’s decision to restrict coverage means only wealthy seniors can afford to pay $26,500 per year out of pocket.

“We ask that CMS provide full and unrestricted Medicare coverage for FDA-approved Alzheimer’s treatments, consistent with its decades-long practice of covering FDA-approved prescription drugs for Medicare bene

In [23]:
article = cnbc_relevant.iloc[4, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

The article discusses the push by Democratic and Republican attorneys general in nearly half of U.S. states to call on Medicare to provide unrestricted coverage of antibody treatments for Alzheimer's disease. The letter was signed by attorneys general from 23 states, the District of Columbia, and two U.S. territories. The letter adds to mounting pressure on the federal Centers for Medicare and Medicaid Services to end a controversial policy that severely restricts access to new drugs such as Eisai and Biogen's Leqembi. The letter acknowledges that Leqembi is associated with certain side effects, such as brain swelling and bleeding, but it says families and their doctors can assess these risks against the benefit of patients being able to recognize their loved ones for a longer period. The letter comes after Medicare rejected a request by the Alzheimer's Association to cover Leqembi without any conditions. The letter also notes that the decision puts older Americans living in rural area

In [24]:
cnbc_score.loc[4, "Relevancy"] = 3
cnbc_score.loc[4, "Accuracy"] = 0
cnbc_score.loc[4, "Summary"] = summary
cnbc_score.loc[4, "Comment"] = "Covers too much of the political and rationales regarding the policy instead of the impact of the policy from an investment perspective. Also attributed things not in letter to the letter."
cnbc_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.cnbc.com/2023/05/02/pfizer-pfe-q1-...,1,4,Pfizer reported first-quarter revenue and adju...,Did not go into detail about the specific merg...
1,https://www.cnbc.com/2023/05/01/scientists-dev...,1,4,The article discusses a new AI system develope...,Did not mention that the system is only in lab...
2,https://www.cnbc.com/2023/04/30/alzheimers-tre...,1,1,capacity to provide the twice-monthly infusio...,Ran out of context window
3,https://www.cnbc.com/2023/05/02/gilead-governm...,1,5,The U.S. government is suing Gilead Sciences f...,Covers the lawsuit and background information ...
4,https://www.cnbc.com/2023/05/01/states-medicar...,0,3,The article discusses the push by Democratic a...,Covers too much of the political and rationale...


In [26]:
cnbc_score.to_csv("gs://article-validation/manual/cnbc-t5-score.csv", index=False)

# Reuters

In [28]:
reuter_df = pd.read_excel("gs://scraped-news-article-data-null/Data_Reuters.xlsx")
reuter_df.head()

,Source,Date,Title,Author,Text,Summary
0,https://www.reuters.com/business/healthcare-ph...,2023-04-28,Pfizer pledge for more equal access to RSV sho...,"Jennifer Rigby, Maggie Fick","LONDON, April 28 (Reuters) - Pfizer Inc (PFE.N...",NaN
1,https://www.reuters.com/business/healthcare-ph...,2023-04-28,US FDA approves Pfizer's next-generation pneum...,Sriparna Roy,April 28 (Reuters) - The U.S. Food and Drug Ad...,NaN
2,https://www.reuters.com/business/healthcare-ph...,2023-04-29,FDA panel backs restricted use of AstraZeneca'...,"Mariam Sunny, Khushi Mandowara, Aditya Samal",A panel of U.S. Food and Drug Administration a...,NaN
3,https://www.reuters.com/business/healthcare-ph...,2023-04-27,"Drugmakers scout for deals, ramp up research s...",Manas Mishra,"Drugmakers including Merck & Co (MRK.N), Astra...",NaN
4,https://www.reuters.com/business/healthcare-ph...,2023-04-27,"EU publishes proposed drug laws overhaul, sett...",Maggie Fick,Brussels on Wednesday published a long-awaited...,Reforms will be biggest changes to EU's medica...


In [29]:
reuter_relevant = reuter_df[["Source", "Text"]]
reuter_relevant.head()

,Source,Text
0,https://www.reuters.com/business/healthcare-ph...,"LONDON, April 28 (Reuters) - Pfizer Inc (PFE.N..."
1,https://www.reuters.com/business/healthcare-ph...,April 28 (Reuters) - The U.S. Food and Drug Ad...
2,https://www.reuters.com/business/healthcare-ph...,A panel of U.S. Food and Drug Administration a...
3,https://www.reuters.com/business/healthcare-ph...,"Drugmakers including Merck & Co (MRK.N), Astra..."
4,https://www.reuters.com/business/healthcare-ph...,Brussels on Wednesday published a long-awaited...


In [30]:
print(reuter_relevant.iloc[0, 1])

LONDON, April 28 (Reuters) - Pfizer Inc (PFE.N) has pledged to deliver critical new medicines more quickly in low-income nations, but its first such vaccine effort faces hurdles likely to delay distribution in poorer countries by several years, global health officials told Reuters.

Pfizer made a commitment on more equitable access last year, following criticism that it prioritized wealthy nations for doses of its COVID-19 shot early in the pandemic. The company says it wants to shorten a timeline in which poorer countries often get vaccines many years after they are available elsewhere.

The drugmaker received a $28 million grant from the Bill and Melinda Gates Foundation in September to support the launch of the respiratory syncytial virus vaccine in poorer countries, where RSV - a common cold-like virus - is much more likely to be lethal for very young children.

Pfizer's maternal RSV vaccine is expected to be approved for use in pregnant women in August in the United States and sev

In [33]:
article = reuter_relevant.iloc[0, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

Pfizer has made a commitment to deliver critical new medicines more quickly in low-income nations, but its first such vaccine effort faces hurdles likely to delay distribution in poorer countries by several years. The company received a $28 million grant from the Bill and Melinda Gates Foundation in September to support the launch of the respiratory syncytial virus vaccine in poorer countries, where RSV is much more likely to be lethal for very young children. Pfizer has yet to take a number of steps needed to make the vaccine available in developing countries, including seeking approval for the shot from the WHO, whose authorization is used by governments that don't have a strong domestic drugs regulator, and by nonprofits - such as the global vaccine alliance Gavi. The Gates grant will be used to fund trials of a multi-dose vial, which are likely to begin next year and may take another year to complete. The company has already reached a partnership for a maternal vaccine in earlier s

In [34]:
reuter_score = reuter_relevant[["Source"]].copy()
reuter_score["Accuracy"] = 1
reuter_score["Relevancy"] = 1
reuter_score["Summary"] = ""
reuter_score["Comment"] = ""
reuter_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.reuters.com/business/healthcare-ph...,1,1,,
1,https://www.reuters.com/business/healthcare-ph...,1,1,,
2,https://www.reuters.com/business/healthcare-ph...,1,1,,
3,https://www.reuters.com/business/healthcare-ph...,1,1,,
4,https://www.reuters.com/business/healthcare-ph...,1,1,,


In [35]:
reuter_score.loc[0, "Relevancy"] = 5
reuter_score.loc[0, "Accuracy"] = 0
reuter_score.loc[0, "Summary"] = summary
reuter_score.loc[0, "Comment"] = "Covered all the parts about the vaccine development and the grant. However, it attributed some of the information for the Gate foundation to Pfizer."
reuter_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.reuters.com/business/healthcare-ph...,0,5,Pfizer has made a commitment to deliver critic...,Covered all the parts about the vaccine develo...
1,https://www.reuters.com/business/healthcare-ph...,1,1,,
2,https://www.reuters.com/business/healthcare-ph...,1,1,,
3,https://www.reuters.com/business/healthcare-ph...,1,1,,
4,https://www.reuters.com/business/healthcare-ph...,1,1,,


In [36]:
print(reuter_relevant.iloc[1, 1])

April 28 (Reuters) - The U.S. Food and Drug Administration (FDA) has approved Pfizer Inc's (PFE.N) next-generation vaccine to prevent pneumococcal disease in children aged six weeks to 17 years, the company said late on Thursday.

The vaccine, Prevnar 20, offers more protection against the disease versus the company's widely-used Prevnar 13 vaccine, by targeting seven more serotypes of the streptococcus pneumoniae bacteria that causes those infections.

The pneumococcal disease can lead to infections in several parts of the body including the lungs, where they can cause pneumonia.

Prevnar 20 has also been approved to prevent a type of ear infection known as otitis media in infants six weeks through five years of age, the company said.

The approval is based on results from mid-stage and late-stage studies that showed the vaccine's safety and tolerability was similar to Prevnar 13.

About 2,000 cases of the disease occur each year in children under 5 years of age in the United States, 

In [37]:
article = reuter_relevant.iloc[1, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

The FDA has approved Pfizer's next-generation vaccine Prevnar 20 to prevent pneumococcal disease in children aged six weeks to 17 years. The vaccine targets seven more serotypes of the streptococcus pneumoniae bacteria that causes the disease, which can lead to infections in several parts of the body including the lungs. Prevnar 20 has also been approved to prevent a type of ear infection known as otitis media in infants six weeks through five years of age. The approval is based on results from mid-stage and late-stage studies that showed the vaccine's safety and tolerability was similar to Prevnar 13. About 2,000 cases of the disease occur each year in children under 5 years of age in the United States, according to the Center for Disease Control and Prevention.



In [38]:
Data 

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.reuters.com/business/healthcare-ph...,0,5,Pfizer has made a commitment to deliver critic...,Covered all the parts about the vaccine develo...
1,https://www.reuters.com/business/healthcare-ph...,1,5,The FDA has approved Pfizer's next-generation ...,Pretty much a paraphrase of the original article
2,https://www.reuters.com/business/healthcare-ph...,1,1,,
3,https://www.reuters.com/business/healthcare-ph...,1,1,,
4,https://www.reuters.com/business/healthcare-ph...,1,1,,


In [39]:
print(reuter_relevant.iloc[2, 1])

A panel of U.S. Food and Drug Administration advisers on Friday voted for the restricted use of British drugmaker AztraZeneca Plc's (AZN.L) experimental treatment, jointly developed with Merck & Co (MRK.N), for a type of prostate cancer.

The FDA panel voted 11-1 with one abstention, in favor of Lynparza, in combination with other medications – abiraterone and prednisone or prednisolone – as a first-line treatment for a type of treatment-resistant prostate cancer in adult patients.

However, the panel recommended the approval to be limited only to patients whose tumors have a mutation in the BCRA genes, and advised against approval beyond this patient population.

Its recommendation comes after Lynparza and similar class of therapies suffered a series of safety setbacks as clinical data suggested that patients do not live as long as those given chemotherapy when given as a second-line therapy.

Lynparza belongs to a class of drugs called PARP inhibitors, which include Clovis Oncology's

In [43]:
article = reuter_relevant.iloc[2, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

The FDA panel has recommended the restricted use of AztraZeneca's experimental treatment for a type of prostate cancer. The panel voted 11-1 with one abstention, in favor of Lynparza, in combination with other medications – abiraterone and prednisone or prednisolone – as a first-line treatment for a type of treatment-resistant prostate cancer in adult patients. However, the panel recommended the approval to be limited only to patients whose tumors have a mutation in the BCRA genes, and advised against approval beyond this patient population. The recommendation comes after Lynparza and similar class of therapies suffered a series of safety setbacks as clinical data suggested that patients do not live as long as those given chemotherapy when given as a second-line therapy. Lynparza belongs to a class of drugs called PARP inhibitors, which include Clovis Oncology's Rubraca and GSK Plc's Zejula. The FDA usually follows the recommendations of its expert panel but is not obligated to do so. 

In [45]:
reuter_score.loc[2, "Relevancy"] = 2
reuter_score.loc[2, "Accuracy"] = 1
reuter_score.loc[2, "Summary"] = summary
reuter_score.loc[2, "Comment"] = "It's just the original article with all the irrelevant parts"
reuter_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.reuters.com/business/healthcare-ph...,0,5,Pfizer has made a commitment to deliver critic...,Covered all the parts about the vaccine develo...
1,https://www.reuters.com/business/healthcare-ph...,1,5,The FDA has approved Pfizer's next-generation ...,Pretty much a paraphrase of the original article
2,https://www.reuters.com/business/healthcare-ph...,1,2,The FDA panel has recommended the restricted u...,It's just the original article with all the ir...
3,https://www.reuters.com/business/healthcare-ph...,1,1,,
4,https://www.reuters.com/business/healthcare-ph...,1,1,,


In [46]:
print(reuter_relevant.iloc[3, 1])

Drugmakers including Merck & Co (MRK.N), AstraZeneca Plc (AZN.L) and AbbVie Inc (ABBV.N) said on Thursday they are open to acquisitions and reported a ramp up in research and development spending as the industry's larger players look for new sources of future revenue.

Several top drugs such as AbbVie flagship Humira have begun to face competition from new rivals or are expected to lose patent protections in the next few years, and deals could be a quick fix to address the loss of revenue from older therapies.

Merck recently announced a $10.8 billion deal to buy Prometheus Biosciences Inc (RXDX.O).

Chief Executive Robert Davis said that deal does not change the company's business development strategy.

"We're focused on building the pipeline, both near and long term, and we do deals across the full spectrum," he said.

Sales of Merck's cancer immunotherapy Keytruda will start losing patent protections in 2028. Meanwhile, it has become the world's top selling prescription drug with fi

In [47]:
article = reuter_relevant.iloc[3, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

Drugmakers including Merck, AstraZeneca, and AbbVie reported first-quarter earnings and reported a ramp up in research and development spending. Some of these companies are facing competition from new rivals or are expected to lose patent protections in the next few years, which could lead to deals to address the loss of revenue from older therapies. Merck recently announced a $10.8 billion deal to buy Prometheus Biosciences, which does not change the company's business development strategy. Sales of Merck's cancer immunotherapy Keytruda will start losing patent protections in 2028, while it has become the world's top selling prescription drug with first-quarter sales of $5.8 billion, exceeding even analysts' lofty estimates. AbbVie reported sales that missed estimates for most treatments, including its blockbuster arthritis drug Humira, and its shares closed down 8%. Amgen slightly increased its forecast for full-year 2023 revenue to $26.2 billion to $27.3 billion, from the previous o

In [48]:
reuter_score.loc[3, "Relevancy"] = 4
reuter_score.loc[3, "Accuracy"] = 1
reuter_score.loc[3, "Summary"] = summary
reuter_score.loc[3, "Comment"] = "It did mention the R&D related information as well as revenue, but also mentioned generic information as well as share prices"
reuter_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.reuters.com/business/healthcare-ph...,0,5,Pfizer has made a commitment to deliver critic...,Covered all the parts about the vaccine develo...
1,https://www.reuters.com/business/healthcare-ph...,1,5,The FDA has approved Pfizer's next-generation ...,Pretty much a paraphrase of the original article
2,https://www.reuters.com/business/healthcare-ph...,1,2,The FDA panel has recommended the restricted u...,It's just the original article with all the ir...
3,https://www.reuters.com/business/healthcare-ph...,1,4,"Drugmakers including Merck, AstraZeneca, and A...",It did mention the R&D related information as ...
4,https://www.reuters.com/business/healthcare-ph...,1,1,,


In [49]:
print(reuter_relevant.iloc[4, 1])

Brussels on Wednesday published a long-awaited draft of its proposed overhaul of laws governing the European Union's pharmaceuticals industry, setting up a tussle with drugmakers which warn they will invest and innovate elsewhere.

The biggest overhaul of existing medical laws in two decades is aimed at ensuring all Europeans have access to both innovative new treatments and generic drugs, and ending huge divergences in access and price between countries, EU Health Commissioner Stella Kyriakides told reporters after publication.

There were few major changes in the proposal from the draft reviewed by Reuters earlier this month, except for details of changes to protections before generic versions of drugs enter the market.

The Commission proposes to cut the length of basic market exclusivity that drugmakers get before generics can enter the market to eight from 10 years.

But it also offers a sweetener for companies: they get two more years of protection if they launch their new medici

In [50]:
article = reuter_relevant.iloc[4, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

Brussels has published a draft of its proposed overhaul of laws governing the European Union's pharmaceuticals industry. The overhaul aims to ensure all Europeans have access to both innovative new treatments and generic drugs, and end huge divergences in access and price between countries. The proposal includes changes to protections before generic versions of drugs enter the market, which could provide access to new medicines to around 70 million more citizens compared to today. Patient and consumer groups generally cheered the proposals, but drugmakers say the reforms will have the opposite effect and result in Europe missing out on the newest treatments. The reforms also aim to prevent drug shortages like those seen this winter with critical antibiotics by requiring companies to notify the EU of possible supply issues earlier. The EU's drug regulator is also streamlined to speed up the time it takes for new treatments to be approved. The final details of the law will be thrashed ou

In [51]:
reuter_score.loc[4, "Relevancy"] = 3
reuter_score.loc[4, "Accuracy"] = 1
reuter_score.loc[4, "Summary"] = summary
reuter_score.loc[4, "Comment"] = "It contains most of the article, and contains too much other information such as the drug shortage"
reuter_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.reuters.com/business/healthcare-ph...,0,5,Pfizer has made a commitment to deliver critic...,Covered all the parts about the vaccine develo...
1,https://www.reuters.com/business/healthcare-ph...,1,5,The FDA has approved Pfizer's next-generation ...,Pretty much a paraphrase of the original article
2,https://www.reuters.com/business/healthcare-ph...,1,2,The FDA panel has recommended the restricted u...,It's just the original article with all the ir...
3,https://www.reuters.com/business/healthcare-ph...,1,4,"Drugmakers including Merck, AstraZeneca, and A...",It did mention the R&D related information as ...
4,https://www.reuters.com/business/healthcare-ph...,1,3,Brussels has published a draft of its proposed...,"It contains most of the article, and contains ..."


In [52]:
print(reuter_relevant.iloc[5, 1])

Drugmaker BeiGene Ltd (6160.HK)said on Thursday the experimental drug it is developing with Swiss firm Novartis AG (NOVN.S) helped extend survival for patients with types of gastric cancer in a late-stage trial.

With the data, BeiGene is hoping the drug could become the first-line of therapy for patients with advanced forms of gastric cancer, or a rare type of cancer that begins in the area where the food pipe and stomach join.

The drug's sales, at their peak, could reach $1.2 billion in China alone, TD Cowen analyst Yaron Werber estimated. The drug, tislelizumab, is currently approved for 10 different cancer indications in China, but is not approved in the United States.

Tislelizumab, in combination with chemotherapy, prolonged survival in patients whose gastric cancer has either spread to other parts of the body, or is in advanced stages. It was also tested in patients with gastro-esophageal junction cancer, the company said.

The drug is currently under review by the U.S. Food an

In [53]:
article = reuter_relevant.iloc[5, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

The article discusses the results of a late-stage trial for BeiGene's experimental drug, tislelizumab, which is being developed with Swiss firm Novartis. The trial showed that the drug helped extend survival for patients with advanced forms of gastric cancer and gastro-esophageal junction cancer. The drug is currently approved for 10 different cancer indications in China, but is not approved in the United States. The drug is currently under review by the U.S. Food and Drug Administration as a second-line therapy for another type of esophageal cancer after prior chemotherapy. BeiGene and Novartis entered into a licensing deal worth up to $2.2 billion in 2021 for tislelizumab. The trial results could impact investment decisions on BeiGene and Novartis based on the potential sales of the drug in China and the potential for it to become the first-line of therapy for patients with advanced forms of gastric cancer or a rare type of cancer that begins in the area where the food pipe and stoma

In [54]:
reuter_score.loc[5, "Relevancy"] = 3
reuter_score.loc[5, "Accuracy"] = 1
reuter_score.loc[5, "Summary"] = summary
reuter_score.loc[5, "Comment"] = "It contains most of the article, and could be shorter just with the trial information"
reuter_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.reuters.com/business/healthcare-ph...,0,5,Pfizer has made a commitment to deliver critic...,Covered all the parts about the vaccine develo...
1,https://www.reuters.com/business/healthcare-ph...,1,5,The FDA has approved Pfizer's next-generation ...,Pretty much a paraphrase of the original article
2,https://www.reuters.com/business/healthcare-ph...,1,2,The FDA panel has recommended the restricted u...,It's just the original article with all the ir...
3,https://www.reuters.com/business/healthcare-ph...,1,4,"Drugmakers including Merck, AstraZeneca, and A...",It did mention the R&D related information as ...
4,https://www.reuters.com/business/healthcare-ph...,1,3,Brussels has published a draft of its proposed...,"It contains most of the article, and contains ..."


In [55]:
print(reuter_relevant.iloc[6, 1])

AstraZeneca plans to spend about $450 million building a factory in China making inhalers to treat "smoker's lung", doubling down on the world's No.2 pharmaceuticals market as the drugmaker predicts only "minimal" sales of its COVID vaccine this year.

China is home to about 100 million patients with chronic obstructive pulmonary disease (COPD), the third leading cause of deaths in the country, the Anglo-Swedish company said.

Smoking and air pollution are the main causes of the illness, sometimes also called emphysema or chronic bronchitis, according to the World Health Organization.

"It's incredibly common in China as people smoke ... but it's not very well diagnosed or treated," said AstraZeneca (AZN.L) CEO Pascal Soriot on a call with media following the release of first-quarter results.

In addition to the site in Qingdao, in the eastern province of Shandong, AstraZeneca is also setting up a research and development centre in Guangzhou with an academic centre to boost diagnosis a

In [56]:
article = reuter_relevant.iloc[6, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

AstraZeneca is investing $450 million in China to build a factory and research and development center to treat "smoker's lung" in the country. The company is also forecasting revenue from China to grow by a low single-digit percentage this year. The move is aimed at tapping into the country's large population of patients with chronic obstructive pulmonary disease (COPD), which is the third leading cause of deaths in the country. The COVID restrictions in China were lifted late last year, reviving demand for healthcare as patients sought treatment and medicines. AstraZeneca CEO Pascal Soriot recently returned from a trip to China, where he saw signs of the country's economy recovering and the government open for collaboration with foreign companies. Soriot also noted that he doesn't see geopolitical tensions between China and Western countries as a challenge for the industry.



In [57]:
reuter_score.loc[6, "Relevancy"] = 4
reuter_score.loc[6, "Accuracy"] = 1
reuter_score.loc[6, "Summary"] = summary
reuter_score.loc[6, "Comment"] = "Talked about the factory, the growth of revenue, and background info on smoker's lung. But, it missed the R&D lab as well as the licensing deal."
reuter_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.reuters.com/business/healthcare-ph...,0,5,Pfizer has made a commitment to deliver critic...,Covered all the parts about the vaccine develo...
1,https://www.reuters.com/business/healthcare-ph...,1,5,The FDA has approved Pfizer's next-generation ...,Pretty much a paraphrase of the original article
2,https://www.reuters.com/business/healthcare-ph...,1,2,The FDA panel has recommended the restricted u...,It's just the original article with all the ir...
3,https://www.reuters.com/business/healthcare-ph...,1,4,"Drugmakers including Merck, AstraZeneca, and A...",It did mention the R&D related information as ...
4,https://www.reuters.com/business/healthcare-ph...,1,3,Brussels has published a draft of its proposed...,"It contains most of the article, and contains ..."


In [58]:
print(reuter_relevant.iloc[7, 1])

The U.S. Food and Drug Administration (FDA) on Friday identified the recall of Avanos Medical Inc's (AVNS.N) artificial airway tube for children and infants as the most serious type, as its use could lead to serious injuries or death.

The medical device maker recalled 1,000 of its Ballard Access Closed Suction System tubes for children in February this year, after it received complaints from customers about cracked openings or manifolds during use of the tubes.

A cracked manifold can cause leaks in the respiratory circuit and can result in the patient receiving insufficient ventilation. The tubes are used for infants and children who are in need of mechanical ventilation.

The recalled devices were distributed between Oct. 20 last year and Jan. 23.

Avanos had received reports of four injuries and one death related to the issue, the FDA said.

The company did not immediately respond to a Reuters request for comment.


In [59]:
article = reuter_relevant.iloc[7, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

The FDA has identified a recall of Avanos Medical's artificial airway tube for children and infants as the most serious type, as its use could lead to serious injuries or death. The company recalled 1,000 of its Ballard Access Closed Suction System tubes for children in February this year after it received complaints from customers about cracked openings or manifolds during use of the tubes. A cracked manifold can cause leaks in the respiratory circuit and can result in the patient receiving insufficient ventilation. The recalled devices were distributed between Oct. 20 last year and Jan. 23. Avanos has received reports of four injuries and one death related to the issue, the FDA said. The company did not immediately respond to a Reuters request for comment.



In [60]:
reuter_score.loc[7, "Relevancy"] = 2
reuter_score.loc[7, "Accuracy"] = 1
reuter_score.loc[7, "Summary"] = summary
reuter_score.loc[7, "Comment"] = "Just the recall and injury/death part would be sufficient."
reuter_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.reuters.com/business/healthcare-ph...,0,5,Pfizer has made a commitment to deliver critic...,Covered all the parts about the vaccine develo...
1,https://www.reuters.com/business/healthcare-ph...,1,5,The FDA has approved Pfizer's next-generation ...,Pretty much a paraphrase of the original article
2,https://www.reuters.com/business/healthcare-ph...,1,2,The FDA panel has recommended the restricted u...,It's just the original article with all the ir...
3,https://www.reuters.com/business/healthcare-ph...,1,4,"Drugmakers including Merck, AstraZeneca, and A...",It did mention the R&D related information as ...
4,https://www.reuters.com/business/healthcare-ph...,1,3,Brussels has published a draft of its proposed...,"It contains most of the article, and contains ..."


In [62]:
reuter_score.to_csv("gs://article-validation/manual/reuter-t5-score.csv", index=False)

# WSJ

In [87]:
wsj_df = pd.read_excel("gs://scraped-news-article-data-null/Data WSJ.xlsx")
wsj_df.head()

,Source,Data,Tittle,Author,Date,Text,Summary
0,WSJ,https://www.wsj.com/articles/health-system-kai...,Health System Kaiser Permanente to Combine Wit...,Anna Wilde Mathews,"April 26, 2023 12:00 pm ET","Kaiser Permanente, a pioneering integrated hea...",Combined nonprofit aims to add more hospital s...
1,WSJ,https://www.wsj.com/articles/kidney-care-speci...,Kidney-Care Specialist Diaverum to Be Sold in ...,Ben Dummett,"April 19, 2023 4:37 am ET",A Mubadala Investment Co.-backed healthcare co...,Mubadala-backed M42 is buying the operator of ...
2,WSJ,https://www.wsj.com/articles/healthcare-lender...,Healthcare Lenders See Increased Demand Amid B...,Brian Gormley,"April 3, 2023 6:00 am ET·",Investment firms that lend to healthcare compa...,Life sciences companies seek alternatives to e...
3,WSJ,https://www.wsj.com/articles/former-aetna-ceo-...,Former Aetna CEO to Take Helm of Health Insure...,Anna Wilde Mathews,"March 28, 2023 6:00 am ET","Mark Bertolini, former chief executive of heal...",Mark Bertolini said he would initially focus o...
4,WSJ,https://www.wsj.com/articles/medical-device-ma...,Medical-Device Makers Gain Amid Return to Norm...,David Wainer,"April 12, 2023 7:00 am ET",Medical-device makers’ stocks are basking in t...,"Stryker, Edwards Lifesciences and Boston Scien..."


In [88]:
wsj_relevant = wsj_df[["Data ", "Text"]].copy()
wsj_relevant = wsj_relevant.rename(columns={"Data ": "Source"})
wsj_relevant.head()

,Source,Text
0,https://www.wsj.com/articles/health-system-kai...,"Kaiser Permanente, a pioneering integrated hea..."
1,https://www.wsj.com/articles/kidney-care-speci...,A Mubadala Investment Co.-backed healthcare co...
2,https://www.wsj.com/articles/healthcare-lender...,Investment firms that lend to healthcare compa...
3,https://www.wsj.com/articles/former-aetna-ceo-...,"Mark Bertolini, former chief executive of heal..."
4,https://www.wsj.com/articles/medical-device-ma...,Medical-device makers’ stocks are basking in t...


In [89]:
print(wsj_relevant.iloc[0, 1])

Kaiser Permanente, a pioneering integrated health system based on the West Coast, has agreed to acquire a Pennsylvania-based hospital operator in a bid to create a new national player in the rapidly consolidating healthcare market. The combination of Kaiser and Geisinger, which must be reviewed by federal and state agencies, would have more than $100 billion in revenue. The subsidiary, called Risant Health, aims to acquire four or five more hospital systems and get to total revenue of $30 billion to $35 billion over the next five years, the systems said. Geisinger Chief Executive Jaewon Ryu would lead Risant. “Kaiser Permanente, through Risant, is really stepping out, putting a stake in the ground, and saying we’re going to partner with these leading health systems,” Kaiser Chief Executive Greg Adams said. Both systems helped model the integration of health insurance, hospital ownership and other operations. The biggest healthcare companies in the U.S., including UnitedHealth Group Inc

In [90]:
article = wsj_relevant.iloc[0, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

Kaiser Permanente has announced plans to acquire Geisinger Health System in a bid to create a new national player in the healthcare market. The combination of Kaiser and Geisinger would have more than $100 billion in revenue and aim to acquire four or five more hospital systems over the next five years. The acquisition is part of a trend of consolidation in the healthcare industry, with major players such as UnitedHealth Group and CVS Health also pursuing similar deals. The combination of Kaiser and Geisinger would be led by Geisinger Chief Executive Jaewon Ryu, who has a track record of integrating healthcare functions. The acquisition is expected to boost revenue and put Kaiser in the conversation with UnitedHealth and CVS Health in terms of developing a national care platform.



In [91]:
wsg_score = wsj_relevant[["Source"]].copy()
wsg_score["Accuracy"] = 1
wsg_score["Relevancy"] = 1
wsg_score["Summary"] = ""
wsg_score["Comment"] = ""
wsg_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.wsj.com/articles/health-system-kai...,1,1,,
1,https://www.wsj.com/articles/kidney-care-speci...,1,1,,
2,https://www.wsj.com/articles/healthcare-lender...,1,1,,
3,https://www.wsj.com/articles/former-aetna-ceo-...,1,1,,
4,https://www.wsj.com/articles/medical-device-ma...,1,1,,


In [92]:
wsg_score.loc[0, "Relevancy"] = 4
wsg_score.loc[0, "Accuracy"] = 1
wsg_score.loc[0, "Summary"] = summary
wsg_score.loc[0, "Comment"] = "Should have mentioned Risant Health. But, it does cover the merger, which is the most important"
wsg_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.wsj.com/articles/health-system-kai...,1,4,Kaiser Permanente has announced plans to acqui...,"Should have mentioned Risant Health. But, it d..."
1,https://www.wsj.com/articles/kidney-care-speci...,1,1,,
2,https://www.wsj.com/articles/healthcare-lender...,1,1,,
3,https://www.wsj.com/articles/former-aetna-ceo-...,1,1,,
4,https://www.wsj.com/articles/medical-device-ma...,1,1,,


In [93]:
print(wsj_relevant.iloc[1, 1])

A Mubadala Investment Co.-backed healthcare company said it agreed to acquire kidney-care provider Diaverum, extending the Abu Dhabi sovereign-wealth fund’s bet on healthcare. M42, a newly launched healthcare operator owned by Mubadala and a partner company, G42, is acquiring Diaverum from U.K.-based buyout firm Bridgepoint Group PLC. BPT 0.17%increase; green up pointing triangle The deal value wasn’t disclosed. The Wall Street Journal earlier reported the transaction values the business at more than $2 billion including debt, according to people familiar with the matter. M42 said it expects to close the deal later this year. A pre-markets primer packed with news, trends and ideas. Plus, up-to-the-minute market data. Diaverum, based in Sweden, oversees a global network of 440 dialysis clinics, almost triple the 155 facilities Bridgepoint inherited when it bought the business in 2007 for an undisclosed amount. Diaverum has also expanded geographically over that period, operating in 23 c

In [94]:
article = wsj_relevant.iloc[1, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

The article discusses the acquisition of Diaverum, a Swedish dialysis provider, by M42, a newly launched healthcare operator owned by Mubadala Investment Co. and a partner company, G42. The deal value of the acquisition is not disclosed, but the Wall Street Journal earlier reported the transaction values the business at more than $2 billion including debt. M42 expects to close the deal later this year.
Diaverum is a Swedish dialysis provider with a global network of 440 dialysis clinics. The company operates in 23 countries and has 40 dialysis facilities in Saudi Arabia, where it operates under a five-year contract with the Saudi Ministry of Health. The size of the dialysis market globally is expected to grow in coming years due to aging populations and the increased incidence of kidney disease related to diabetes and other conditions.
M42 has an existing base in the region and Mubadala's backing, which gave it an advantage over other bidders in the sale process. Bridgepoint Group atte

In [95]:
wsg_score.loc[1, "Relevancy"] = 4
wsg_score.loc[1, "Accuracy"] = 1
wsg_score.loc[1, "Summary"] = summary
wsg_score.loc[1, "Comment"] = "Contains information about the merger. But, the last paragraph is not really needed."
wsg_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.wsj.com/articles/health-system-kai...,1,4,Kaiser Permanente has announced plans to acqui...,"Should have mentioned Risant Health. But, it d..."
1,https://www.wsj.com/articles/kidney-care-speci...,1,4,The article discusses the acquisition of Diave...,"Contains information about the merger. But, th..."
2,https://www.wsj.com/articles/healthcare-lender...,1,1,,
3,https://www.wsj.com/articles/former-aetna-ceo-...,1,1,,
4,https://www.wsj.com/articles/medical-device-ma...,1,1,,


In [96]:
print(wsj_relevant.iloc[2, 1])

Investment firms that lend to healthcare companies are fielding increased interest from entrepreneurs as the banking crisis unfolds. Interest in debt was rising before Silicon Valley Bank’s collapse. But uncertainty stemming from banking industry turmoil has sharpened startups’ focus on boosting their balance sheets with less equity financing, which has grown more costly, investors said. New York-based life sciences firm Catalio Capital Management makes venture-capital investments and backs publicly traded companies. In November, Catalio said it had raised more than $85 million for its first special-situations fund, a vehicle targeting debt and structured-equity deals for small and midsize private companies. Catalio has invested about half that pool and intends to raise a second special-situations fund, well ahead of schedule, because of demand, said George Petrocheilos, co-founder and a managing partner of the firm. Catalio hasn’t determined the amount of the next special-situations f

In [97]:
article = wsj_relevant.iloc[2, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

The banking crisis has led to increased interest in debt financing for healthcare companies, particularly in the wake of Silicon Valley Bank's collapse. New York-based life sciences firm Catalio Capital Management has raised a special-situations fund targeting debt and structured-equity deals for small and midsize private companies, and plans to raise a second fund ahead of schedule due to demand. OrbiMed Advisors has a dedicated credit and royalty fund and is finding that more large companies with well-established revenue bases are showing interest in credit and royalty financing. Runway Growth Capital LLC makes about 30% of its loans in healthcare, and it is offering larger loans with flexible terms to healthcare companies. The banking crisis has also led to increased interest in debt financing for late-stage and growth-stage companies, particularly in the wake of the collapse of Silicon Valley Bank.



In [98]:
wsg_score.loc[2, "Relevancy"] = 4
wsg_score.loc[2, "Accuracy"] = 1
wsg_score.loc[2, "Summary"] = summary
wsg_score.loc[2, "Comment"] = "Does capture the funding situation post SVB, but missed the examples on the health industry companies"
wsg_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.wsj.com/articles/health-system-kai...,1,4,Kaiser Permanente has announced plans to acqui...,"Should have mentioned Risant Health. But, it d..."
1,https://www.wsj.com/articles/kidney-care-speci...,1,4,The article discusses the acquisition of Diave...,"Contains information about the merger. But, th..."
2,https://www.wsj.com/articles/healthcare-lender...,1,4,The banking crisis has led to increased intere...,"Does capture the funding situation post SVB, b..."
3,https://www.wsj.com/articles/former-aetna-ceo-...,1,1,,
4,https://www.wsj.com/articles/medical-device-ma...,1,1,,


In [99]:
print(wsj_relevant.iloc[3, 1])

Mark Bertolini, former chief executive of health insurance giant Aetna Inc. and hedge fund Bridgewater Associates, will take the helm of Oscar Health Inc. as it seeks to turn a profit and carve out a role as a technology supplier in the healthcare industry. Mr. Bertolini, 66 years old, will take the post effective next Monday, the company said. He will succeed Mario Schlosser, 44, who co-founded Oscar in 2012 with Joshua Kushner and will take the new title of president of technology, reporting to Mr. Bertolini. Mr. Schlosser will also remain on Oscar’s board, which Mr. Bertolini will join, the company said. Oscar is expected to announce the changes Tuesday. Mr. Bertolini said he would initially focus on ensuring Oscar meets its goal of having a profitable insurance business this year and full-company profitability in 2024. Beyond that, he said, he thinks the company can supply technology tools to help doctors and health systems interact with patients and manage their care, a different 

In [100]:
article = wsj_relevant.iloc[3, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

Mark Bertolini, former CEO of Aetna and hedge fund Bridgewater Associates, will take the helm of Oscar Health as it seeks to turn a profit and carve out a role as a technology supplier in the healthcare industry. Bertolini will take the post effective next Monday, and will succeed Mario Schlosser, who co-founded Oscar in 2012 with Joshua Kushner and will take the new title of president of technology, reporting to Bertolini. Mr. Schlosser will also remain on Oscar's board, which Bertolini will join. Bertolini will initially focus on ensuring Oscar meets its goal of having a profitable insurance business this year and full-company profitability in 2024. He believes the company can supply technology tools to help doctors and health systems interact with patients and manage their care, a different approach than some large insurers that have been more focused on buying up doctor groups, clinics, and other assets. This would pit Oscar against many of the largest companies in the industry, in

In [101]:
wsg_score.loc[3, "Relevancy"] = 3
wsg_score.loc[3, "Accuracy"] = 1
wsg_score.loc[3, "Summary"] = summary
wsg_score.loc[3, "Comment"] = "Does cover the change in leadership, but too much background"
wsg_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.wsj.com/articles/health-system-kai...,1,4,Kaiser Permanente has announced plans to acqui...,"Should have mentioned Risant Health. But, it d..."
1,https://www.wsj.com/articles/kidney-care-speci...,1,4,The article discusses the acquisition of Diave...,"Contains information about the merger. But, th..."
2,https://www.wsj.com/articles/healthcare-lender...,1,4,The banking crisis has led to increased intere...,"Does capture the funding situation post SVB, b..."
3,https://www.wsj.com/articles/former-aetna-ceo-...,1,3,"Mark Bertolini, former CEO of Aetna and hedge ...","Does cover the change in leadership, but too m..."
4,https://www.wsj.com/articles/medical-device-ma...,1,1,,


In [102]:
print(wsj_relevant.iloc[4, 1])

Medical-device makers’ stocks are basking in the good times. That is a marked shift from the turbulence of recent years, when device makers contended with inflationary pressures and a drop in medical procedures as hospitals faced staffing shortages and many patients deferred surgery during the pandemic. While the industry was far from the only sector to be affected by Covid-19, it suffered more than large-cap healthcare peers such as drugmakers and insurers. Those industries were relatively immune to the pandemic’s economic shocks, with insurers actually benefiting from a decline in surgical procedures. Now, that picture is starting to reverse, and device makers are looking more resilient than their healthcare counterparts. Not only have cost pressures such as parts shortages started to abate, but procedure volumes are rebounding as Covid-19 restrictions loosen worldwide. And while pharma and insurers continue to face political pressure in Washington, device makers are relatively insul

In [103]:
article = wsj_relevant.iloc[4, 1]
full_prompt = prompt + article
summary = generate(full_prompt, temperature=0)
print(summary)

Medical-device makers' stocks are up around 10% or more so far this year, indicating a return to normal in the industry. The sector was affected more by the pandemic than other large-cap healthcare peers, but cost pressures and procedure volumes are rebounding. Some companies, such as Boston Scientific, are benefiting from a new product cycle and are expected to raise full-year guidance. The industry is still expensive, but strong revenue growth means that investors are likely to remain positive on the sector. Investors will be paying close attention when industry behemoth UnitedHealth Group reports first-quarter earnings before the bell on Friday.



In [104]:
wsg_score.loc[4, "Relevancy"] = 5
wsg_score.loc[4, "Accuracy"] = 1
wsg_score.loc[4, "Summary"] = summary
wsg_score.loc[4, "Comment"] = "Captures the big trend without going in too much on the details"
wsg_score.head()

,Source,Accuracy,Relevancy,Summary,Comment
0,https://www.wsj.com/articles/health-system-kai...,1,4,Kaiser Permanente has announced plans to acqui...,"Should have mentioned Risant Health. But, it d..."
1,https://www.wsj.com/articles/kidney-care-speci...,1,4,The article discusses the acquisition of Diave...,"Contains information about the merger. But, th..."
2,https://www.wsj.com/articles/healthcare-lender...,1,4,The banking crisis has led to increased intere...,"Does capture the funding situation post SVB, b..."
3,https://www.wsj.com/articles/former-aetna-ceo-...,1,3,"Mark Bertolini, former CEO of Aetna and hedge ...","Does cover the change in leadership, but too m..."
4,https://www.wsj.com/articles/medical-device-ma...,1,5,Medical-device makers' stocks are up around 10...,Captures the big trend without going in too mu...


In [106]:
wsg_score.to_csv("gs://article-validation/manual/wsj-t5-score.csv", index=False)